In [4]:
import joblib

from data import load_train_test_sets

In [2]:
final_model = joblib.load('../models/final_model.clf')
final_model

Pipeline(steps=[('column_transformer',
                 ColumnTransformer(transformers=[('description_bow',
                                                  CountVectorizer(max_df=0.05,
                                                                  min_df=500,
                                                                  stop_words={'english'}),
                                                  'Description'),
                                                 ('title_bow',
                                                  CountVectorizer(max_df=0.1,
                                                                  min_df=500,
                                                                  stop_words={'english'}),
                                                  'Name')])),
                ('svd', TruncatedSVD(n_components=50)),
                ('classifier', LogisticRegression(random_state=42))])

In [5]:
X_train, y_train, X_test, y_test = load_train_test_sets()

Proportion of popular books in the dataset: 0.07
75122


In [6]:
y_pred = final_model.predict(X_train)
X_train[y_pred != y_train].head()

/var/folders/1h/gxksx08n6xgb0ntrfqt6hl6w0000gn/T/ipykernel_4066/401241314.py:2: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  X_train[y_pred != y_train].head()


ValueError: Unable to coerce to Series, length must be 7: given 958623